# Decision Tree Machine Learning Model

### Read in csv file

In [3]:
import csv
import numpy as np
import pandas as pd

In [4]:
filename = '50007-2022/train_tfidf_features.csv'
train_features = pd.read_csv (filename, header=0)

print(train_features.shape)

(17184, 5002)


In [5]:
print(train_features.head(10))

   id  label    0    1    2    3    4    5    6    7  ...  4990  4991  4992  \
0   1      1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
1   2      0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
2   3      1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
3   4      0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4   5      1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
5   6      0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
6   7      0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
7   8      1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
8   9      1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
9  10      1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   

   4993  4994  4995  4996  4997  4998  4999  
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1   0.0   0.0   0.0   0.0   0.0   0.0 

In [6]:
train_features.set_index('id', inplace=True, drop=True)
print(train_features.head(10))

    label    0    1    2    3    4    5    6    7    8  ...  4990  4991  4992  \
id                                                      ...                     
1       1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
2       0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
3       1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4       0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
5       1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
6       0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
7       0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
8       1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
9       1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
10      1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   

    4993  4994  4995  4996 

### Node Object

In [7]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        ## which feature to use for splitting dataset
        self.feature_index = feature_index
        ## threshold value to split dataset
        self.threshold = threshold
        ## root node of the left subtree
        self.left = left
        ## root node of the left subtree
        self.right = right
        ## information gained from the this node
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

### Decision Tree Classifier Object

In [24]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        ## if number of data samples in dataset to split using build_tree cannot be 
        ## smaller than min_samples_split (default to 2 points)
        self.min_samples_split = min_samples_split
        ## building a tree with ever increasing depth as it recurses maximum depth of 
        ## a branch cannot exceed max_depth(default 3 levels if root node is included)

        # TODO 
        # As the dataset is big, it is prone to overfitting, we are unlikely to 
        # split the dataset into less than min_samples_split. Hence, it is likely we should
        # instead determin the max_depth of the tree. 
        # Hence making max_depth our HYPERPARAMETER
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # only split if sample is large enough or max depth is not exceeded
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            # returns a dictionary that characterises the best_split (with the most information gain) 
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recursion: build_tree on the left, left_subtree is the root node of the left subtree
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recursion: build_tree on the right, right_subtree is the root node of right subtree
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return current decision node (with inputs left node, right node and node attributes)
                # not a leaf so no value
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        # only leaf node has value
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        # max_info_gain from any decision threshold boundary to be negative infinity (monotonically increasing)  
        max_info_gain = -float("inf")
        
        # For each feature i
        for feature_index in range(num_features):
            # Finding unique values for a particular feature
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # For every unique value/possible threshold in feature i, calculate the information gain
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y)
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    # TODO
    # Alternatively, entropy can be used instead of gini_index but because entropy is more computationally
    # intensive so it was not used. Loss function can be considered a HYPERPARAMETER. Entropy criterion
    # only performs marginally better but might be worth exploring
    def information_gain(self, parent, l_child, r_child):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
       
        return gain
    
    def gini_index(self, y):
        ''' function to compute gini index '''

        # Formula of gini index is 1 - summation((probability of class i)**2)
        p1_sq = (len(y[y == 1]) / len(y))**2
        p0_sq = (len(y[y == 0]) / len(y))**2
        gini = 1- p1_sq - p0_sq

        return gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        # value of leaf value == most commonly found class in the leaf within data 
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("Feature: "+str(tree.feature_index), "<=", tree.threshold, ",info_gain: ", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, node):
        ''' function to predict a single data point '''
        
        # If leaf node
        if node.value!=None: 
            return node.value
        # If not leaf node, need to determine whether to continue on the left or right node
        feature_val = x[node.feature_index]
        if feature_val<=node.threshold:
            return self.make_prediction(x, node.left)
        else:
            return self.make_prediction(x, node.right)

### Train-Test split

In [34]:
#TODO 
#Need to use the dataset with reduced dimensions
X = train_features.iloc[:, 10:20].values
Y = train_features.iloc[:, 0].values.reshape(-1,1)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

## Fit the model

In [31]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=6)
classifier.fit(X_train,Y_train)
classifier.print_tree()

Feature: 1 <= 0.0 ,info_gain:  0.0002817556815680744
 left:Feature: 8 <= 0.0 ,info_gain:  0.00011559327255450391
  left:Feature: 5 <= 0.5128646382061476 ,info_gain:  0.00011292045727268052
    left:Feature: 5 <= 0.3070285320983342 ,info_gain:  0.0001859417935706209
        left:Feature: 5 <= 0.2922486917420209 ,info_gain:  0.00013013016436252967
                left:Feature: 5 <= 0.0 ,info_gain:  0.0001039601168097426
                                left:Feature: 0 <= 0.4082405717900109 ,info_gain:  8.325308761020223e-05
                                                                left:0.0
                                                                right:0.0
                                right:0.0
                right:Feature: 5 <= 0.2994042317448218 ,info_gain:  0.05333333333333318
                                left:1.0
                                right:Feature: 5 <= 0.3019129309487226 ,info_gain:  0.4444444444444445
                                                    

### Test the model

In [33]:
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.6045970322956067

### Analysis

## TODO: Do cross validation so that machine learning model is performs more consistently. 